In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
gene_x_sample = pd.read_table(
    PATH_DICT['gene_x_sample_file_path'],
    index_col=0,
)

target_x_sample = pd.read_table(
    PATH_DICT['target_x_sample_file_path'],
    index_col=0,
)

In [ ]:
def compute_log_ratio(
    logged_values,
    target,
):
    
    return logged_values[target == 1].mean() - logged_values[target == 0].mean()

In [ ]:
make_match_panel_keyword_arguments = dict(
    n_job=N_JOB,
    target_ascending=TARGET_ASCENDING,
    n_sampling=N_SAMPLING,
    n_permutation=N_PERMUTATION,
    target_type=TARGET_TYPE,
    plot_std=PLOT_STD,
)

for target_name, target_values in target_x_sample.iterrows():
    
    output_directory_path = '{}/{}'.format(
        PATH_DICT['differentially_expressed_gene_directory_path'],
        target_name,
    )
    
    ccal.establish_path(
        output_directory_path,
        'directory',
    )
    
    target_values = target_values[target_values != -1]
    
    score_moe_p_value_fdr = ccal.make_match_panel(
        target_values, 
        gene_x_sample,
        extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
        title='All',
        file_path_prefix='{}/all'.format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )
    
    score_moe_p_value_fdr_ = ccal.make_match_panel(
        target_values, 
        gene_x_sample.reindex(GENE_TO_PEEK),
        score_moe_p_value_fdr=score_moe_p_value_fdr,
        extreme_feature_threshold=None,
        title='Peek',
        file_path_prefix='{}/peek'.format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )

    score_moe_p_value_fdr__log_ratio = ccal.make_match_panel(
        target_values, 
        gene_x_sample,
        match_function=compute_log_ratio,
        extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
        title='All (Log Ratio)',
        file_path_prefix='{}/all__log_ratio'.format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )
    
    score_moe_p_value_fdr__log_ratio_ = ccal.make_match_panel(
        target_values, 
        gene_x_sample.reindex(GENE_TO_PEEK),
        score_moe_p_value_fdr=score_moe_p_value_fdr__log_ratio,
        extreme_feature_threshold=None,
        title='Peek (Log Ratio)',
        file_path_prefix='{}/peek__log_ratio'.format(output_directory_path),
        **make_match_panel_keyword_arguments,
    )